## IOU Testing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from PyHa.IsoAutio import generate_automated_labels
from PyHa.statistics import clip_IoU, clip_IoU_lin, clip_IoU_skip, matrix_IoU_Scores
from PyHa.statistics import automated_labeling_statistics

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import time

In [19]:
output_df = {
    "FUNCTION": [],
    "TECHNIQUE": [],
    "DURATION": [],
    "ITERATIONS": [],
    "AVERAGE": []
}
path =  "../acoustic-id-test-data/TEST/"
path2 = "./ScreamingPiha_Manual_Labels.csv"

In [20]:
isolation_parameters = {
    "model" : "microfaune",
    "technique" : "chunk",
    "threshold_type" : "median",
    "threshold_const" : 2.0,
    "threshold_min" : 0.0,
    "chunk_size" : 5.0,
    "window_size" : 2.0
}

manual_df_piha = pd.read_csv(path2)
automated_df_piha = generate_automated_labels(path, isolation_parameters)
#manual_df = pd.read_csv(path2)
#automated_df = generate_automated_labels(path, isolation_parameters)

C:\Users\saman\OneDrive\Documents\GitHub\PyHa\PyHa\microfaune_package\microfaune\audio.py:23: WavFileWarning: Reached EOF prematurely; finished at 61440 bytes, expected 11782700 bytes from header.
  fs, data = wavfile.read(path)


In [19]:
def call_IoU(f, automated_df, manual_df):
    start = time.time()
    matrix = f(automated_df, manual_df)
    print('time:', int((time.time() - start) * 10) / 10.0, 'seconds')
    return matrix

# double_scalars error comes from dividing two very small numbers
# this is handled by converting NaN values in the IoU matrix to 0 when it is returned
print("\nskip")
lin = call_IoU(clip_IoU_skip, automated_df, manual_df)
display(matrix_IoU_Scores(lin, manual_df, 0.5))

print("linalg")
orig = call_IoU(clip_IoU_lin, automated_df, manual_df)
display(matrix_IoU_Scores(orig, manual_df, 0.5))

print("\nlinalg + skip")
slight = call_IoU(clip_IoU, automated_df, manual_df)
display(matrix_IoU_Scores(slight, manual_df, 0.5))

#note that although it says screamingpiha1 (since clip_IoU is supposed to take in one clip), it is for the whole dataset


skip
time: 286.4 seconds


,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,./TEST/,ScreamingPiha1.wav,bird,12,242,82,0.1277,0.0472,0.0689


linalg


C:\Users\saman\OneDrive\Documents\GitHub\PyHa\PyHa\statistics.py:533: RuntimeWarning: invalid value encountered in double_scalars
  IoU_Matrix[i][j] /= np.sum(np.logical_or(human_arr[i], bot_arr[j]))


time: 171.8 seconds


,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,./TEST/,ScreamingPiha1.wav,bird,12,242,82,0.1277,0.0472,0.0689



linalg + skip
time: 23.4 seconds


,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,./TEST/,ScreamingPiha1.wav,bird,12,242,82,0.1277,0.0472,0.0689


## Testing automated_labeling_statistics

In [21]:
def call_stats(automated_df, manual_df, stats_type="IoU", threshold=0.5):
    start = time.time()
    stats_df = automated_labeling_statistics(automated_df,manual_df,stats_type,threshold)
    print(stats_type, 'time:\t', int((time.time() - start) * 10) / 10.0, 'seconds\n')
    return stats_df

stats_df_skip = call_stats(automated_df_piha, manual_df_piha, stats_type="IoU-skip")
stats_df_lin = call_stats(automated_df_piha, manual_df_piha, stats_type="IoU-lin")
stats_df_lin_skip = call_stats(automated_df_piha, manual_df_piha, stats_type="IoU-lin-skip")

IoU-skip time:	 49.9 seconds

IoU-lin time:	 31.1 seconds

IoU-lin-skip time:	 5.6 seconds



In [22]:
display(stats_df_skip)
display(stats_df_lin)
display(stats_df_lin_skip)

,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,./TEST/,ScreamingPiha1.wav,bird,1,12,6,0.1429,0.0769,0.1000
1,./TEST/,ScreamingPiha10.wav,bird,0,40,18,0.0000,0.0000,0.0000
2,./TEST/,ScreamingPiha11.wav,bird,1,34,12,0.0769,0.0286,0.0417
3,./TEST/,ScreamingPiha2.wav,bird,0,20,6,0.0000,0.0000,0.0000
4,./TEST/,ScreamingPiha3.wav,bird,0,6,2,0.0000,0.0000,0.0000
5,./TEST/,ScreamingPiha4.wav,bird,0,8,3,0.0000,0.0000,0.0000
6,./TEST/,ScreamingPiha5.wav,bird,1,29,10,0.0909,0.0333,0.0487
7,./TEST/,ScreamingPiha6.wav,bird,0,19,15,0.0000,0.0000,0.0000
8,./TEST/,ScreamingPiha7.wav,bird,1,50,0,1.0000,0.0196,0.0384
9,./TEST/,ScreamingPiha8.wav,bird,0,9,9,0.0000,0.0000,0.0000


,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,./TEST/,ScreamingPiha1.wav,bird,1,12,6,0.1429,0.0769,0.1000
1,./TEST/,ScreamingPiha10.wav,bird,0,40,18,0.0000,0.0000,0.0000
2,./TEST/,ScreamingPiha11.wav,bird,1,34,12,0.0769,0.0286,0.0417
3,./TEST/,ScreamingPiha2.wav,bird,0,20,6,0.0000,0.0000,0.0000
4,./TEST/,ScreamingPiha3.wav,bird,0,6,2,0.0000,0.0000,0.0000
5,./TEST/,ScreamingPiha4.wav,bird,0,8,3,0.0000,0.0000,0.0000
6,./TEST/,ScreamingPiha5.wav,bird,1,29,10,0.0909,0.0333,0.0487
7,./TEST/,ScreamingPiha6.wav,bird,0,19,15,0.0000,0.0000,0.0000
8,./TEST/,ScreamingPiha7.wav,bird,1,50,0,1.0000,0.0196,0.0384
9,./TEST/,ScreamingPiha8.wav,bird,0,9,9,0.0000,0.0000,0.0000


,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,./TEST/,ScreamingPiha1.wav,bird,1,12,6,0.1429,0.0769,0.1000
1,./TEST/,ScreamingPiha10.wav,bird,0,40,18,0.0000,0.0000,0.0000
2,./TEST/,ScreamingPiha11.wav,bird,1,34,12,0.0769,0.0286,0.0417
3,./TEST/,ScreamingPiha2.wav,bird,0,20,6,0.0000,0.0000,0.0000
4,./TEST/,ScreamingPiha3.wav,bird,0,6,2,0.0000,0.0000,0.0000
5,./TEST/,ScreamingPiha4.wav,bird,0,8,3,0.0000,0.0000,0.0000
6,./TEST/,ScreamingPiha5.wav,bird,1,29,10,0.0909,0.0333,0.0487
7,./TEST/,ScreamingPiha6.wav,bird,0,19,15,0.0000,0.0000,0.0000
8,./TEST/,ScreamingPiha7.wav,bird,1,50,0,1.0000,0.0196,0.0384
9,./TEST/,ScreamingPiha8.wav,bird,0,9,9,0.0000,0.0000,0.0000


### Importing labels

In [3]:
path_to_audio_files = "../acoustic-id-test-data/subset/" 
path_to_ground_truth = "./025_DSC180_subset_manual.csv"

In [4]:
# generate a csv with just the labels for specified files
import os
files = os.listdir(path_to_audio_files)
dscdf = pd.read_csv(path_to_ground_truth)
dscdf = dscdf[dscdf["IN FILE"].isin(files)]
dscdf.to_csv("025_DSC180_subset_manual.csv")
path_to_ground_truth = "025_DSC180_subset_manual.csv"

In [5]:
manual_df = pd.read_csv(path_to_ground_truth)
isolation_parameters = {
    "model" : "microfaune",
    "technique" : "chunk",
    "threshold_type" : "median",
    "threshold_const" : 2.0,
    "threshold_min" : 0.0,
    "chunk_size" : 3.0,
    "window_size" : 2.0
}
#automated_df = generate_automated_labels(path_to_audio_files, isolation_parameters)
#automated_df.to_csv("025_DSC180_subset_automated.csv")
automated_df = pd.read_csv("025_DSC180_subset_automated.csv")

In [15]:
manual_df["FOLDER"] = path_to_audio_files
manual_df

,Unnamed: 0.1,Unnamed: 0,IN FILE,CLIP LENGTH,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,FOLDER
0,0,5,Aburria-aburri-387212.wav,84.480,0,3,44100,bird,../acoustic-id-test-data/subset/
1,1,6,Aburria-aburri-387212.wav,84.480,3,3,44100,bird,../acoustic-id-test-data/subset/
2,2,7,Aburria-aburri-387212.wav,84.480,30,3,44100,bird,../acoustic-id-test-data/subset/
3,3,8,Aburria-aburri-387212.wav,84.480,33,3,44100,bird,../acoustic-id-test-data/subset/
4,4,9,Aburria-aburri-387212.wav,84.480,75,3,44100,bird,../acoustic-id-test-data/subset/
...,...,...,...,...,...,...,...,...,...
5491,5491,22015,Zonotrichia-capensis-232609.wav,98.856,72,3,32000,bird,../acoustic-id-test-data/subset/
5492,5492,22016,Zonotrichia-capensis-232609.wav,98.856,78,3,32000,bird,../acoustic-id-test-data/subset/
5493,5493,22017,Zonotrichia-capensis-232609.wav,98.856,84,3,32000,bird,../acoustic-id-test-data/subset/
5494,5494,22018,Zonotrichia-capensis-232609.wav,98.856,87,3,32000,bird,../acoustic-id-test-data/subset/


In [16]:
%reload_ext autoreload
def call_stats(automated_df, manual_df, stats_type="IoU", threshold=0.5):
    start = time.time()
    stats_df = automated_labeling_statistics(automated_df,manual_df,stats_type, threshold)
    print('time:', int((time.time() - start) * 10) / 10.0, 'seconds')
    return stats_df

stats_df_bn_lin_skip = call_stats(automated_df, manual_df, stats_type="IoU-lin-skip")

Processed 50 clips in 25.8 seconds
Processed 100 clips in 24.4 seconds
Processed 150 clips in 20.5 seconds
Processed 200 clips in 25.4 seconds
Processed 250 clips in 17.9 seconds
Processed 300 clips in 15.5 seconds
Processed 350 clips in 30.9 seconds
Processed 400 clips in 42.9 seconds
Processed 450 clips in 26.0 seconds
Processed 500 clips in 31.6 seconds
Processed 550 clips in 19.7 seconds
Something went wrong with 1 clips out of 596 clips
time: 337.7 seconds


In [18]:
stats_df_bn_lin = call_stats(automated_df, manual_df, stats_type="IoU-lin")

Processed 50 clips in 78.6 seconds
Processed 100 clips in 98.4 seconds
Processed 150 clips in 108.0 seconds
Processed 200 clips in 152.7 seconds
Processed 250 clips in 92.4 seconds
Processed 300 clips in 73.0 seconds
Processed 350 clips in 303.4 seconds
Processed 400 clips in 290.5 seconds
Processed 450 clips in 174.1 seconds
Processed 500 clips in 158.0 seconds
Processed 550 clips in 101.4 seconds
Something went wrong with 1 clips out of 596 clips
time: 2221.1 seconds


In [17]:
stats_df_bn_lin_skip

,FOLDER,IN FILE,MANUAL ID,TRUE POSITIVE,FALSE NEGATIVE,FALSE POSITIVE,PRECISION,RECALL,F1
0,../acoustic-id-test-data/subset/,Aburria-aburri-387212.wav,bird,4,2,16,0.2000,0.6667,0.3077
1,../acoustic-id-test-data/subset/,Accipiter-collaris-260335.wav,bird,10,0,6,0.6250,1.0000,0.7692
2,../acoustic-id-test-data/subset/,Accipiter-collaris-260336.wav,bird,4,0,2,0.6667,1.0000,0.8000
3,../acoustic-id-test-data/subset/,Accipiter-striatus-497793.wav,bird,1,0,1,0.5000,1.0000,0.6667
4,../acoustic-id-test-data/subset/,Actitis-macularius-257516.wav,bird,7,0,1,0.8750,1.0000,0.9333
...,...,...,...,...,...,...,...,...,...
590,../acoustic-id-test-data/subset/,Zebrilus-undulatus-217926.wav,bird,2,5,2,0.5000,0.2857,0.3636
591,../acoustic-id-test-data/subset/,Zimmerius-cinereicapilla-258673.wav,bird,4,0,4,0.5000,1.0000,0.6667
592,../acoustic-id-test-data/subset/,Zimmerius-gracilipes-258668.wav,bird,4,0,0,1.0000,1.0000,1.0000
593,../acoustic-id-test-data/subset/,Zonotrichia-capensis-232609.wav,bird,24,0,9,0.7273,1.0000,0.8421


In [ ]:
stats_df_bn_lin

## Verification

In [10]:
print(len(slight), 'x', len(slight[0]))
print(len(lin), 'x', len(lin[0]))

254 x 93
254 x 93


In [12]:
print(slight)

[[0.4259 0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.17   ... 0.     0.     0.    ]
 [0.2    0.     0.     ... 0.     0.     0.    ]
 ...
 [0.     0.     0.     ... 0.0228 0.2939 0.    ]
 [0.     0.     0.     ... 0.     0.4122 0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]]


In [37]:
slight.tolist()
print(*slight, sep="\n")

[0.4259 0.     0.     0.     0.     0.     0.     0.4259 0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.4259 0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.4259 0.
 0.     0.     0.     0.     0.4259 0.     0.4259 0.     0.     0.4259
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.4259 0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.4259 0.     0.     0.
 0.     0.     0.     0.     0.     0.4259 0.     0.     0.     0.
 0.     0.     0.    ]
[0.     0.     0.17   0.     0.     0.     0.     0.     0.     0.17
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.17   0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.17   0.     0.     0.     0.     0.     0.     0.     0.2389 0.
 0.     0.17   0.     0.     0.  

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.     0.     0.     0.     0.     0.     0.7505 0.     0.     0.
 0.     0.     0.     0.7505 0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.7505 0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.7505 0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.7505 0.     0.     0.     0.
 0.     0.     0.     0.     0

In [28]:
lin.tolist()
print(*lin, sep="\n")

[0.4259 0.     0.     0.     0.     0.     0.     0.4259 0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.4259 0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.4259 0.
 0.     0.     0.     0.     0.4259 0.     0.4259 0.     0.     0.4259
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.4259 0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.4259 0.     0.     0.
 0.     0.     0.     0.     0.     0.4259 0.     0.     0.     0.
 0.     0.     0.    ]
[0.     0.     0.17   0.     0.     0.     0.     0.     0.     0.17
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.17   0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.17   0.     0.     0.     0.     0.     0.     0.     0.2389 0.
 0.     0.17   0.     0.     0.  

 0.    0.342 0.    0.    0.    0.    0.    0.    0.   ]
[0.048  0.     0.     0.     0.     0.     0.     0.048  0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.048  0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.048  0.
 0.     0.     0.     0.     0.048  0.     0.048  0.     0.     0.048
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.048  0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.3448 0.048  0.     0.     0.
 0.     0.     0.     0.     0.     0.048  0.     0.     0.     0.
 0.     0.     0.    ]
[0.118 0.    0.    0.    0.    0.    0.    0.118 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.118 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.118 0.    0.    0.    0.    0.    0.118 0.    0.118 0.
 0.    0.118 0.    0.    0.    0.  

In [45]:
# prints the coords of any differences
import numpy as np
lin = np.nan_to_num(lin)
differences = slight - lin
d = np.nonzero(differences)
d0 = d[0].tolist()
d1 = d[1].tolist()

for i in range(len(d0)):
    print(d0[i], d1[i])